<a href="https://colab.research.google.com/github/NaadineKhaaled/Face-Recognition/blob/main/FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset has 10 images per 40 subjects. Every image is a grayscale
image of size 92x112.

Generate the Data Matrix and the Label vector 
a. Convert every image into a vector of 10304 values corresponding to the
image size.

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import math 
from math import sqrt
import cv2
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import seaborn as sn
import sys
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from PIL import Image
from scipy import linalg



1. Downloading the Dataset

2. Generate the Data Matrix and the Label vector 

3. Split the Dataset into Training and Test sets

In [ ]:
BASE = '/content/drive/MyDrive/Colab Notebooks/FaceRecognition/att-database-of-faces'
Base2 = '/content/drive/MyDrive/Colab Notebooks/FaceRecognition/Flowerpgm'
matrix = []
new_matrix = []#Q7 iii
matrix_Mix = []#Q7
training=[]
testing=[]
trainingLabel=[]
testingLabel=[]
trainingLabel_Mix= [] #Q7
testingLabel_Mix = [] #Q7
new_TrainingLabel = []#Q7 iii
new_TestingLabel = []#Q7 iii
count=0

for i in range(1,41):
  for j in range(1,11):
    img = cv2.imread(BASE + '/s'+ str(i)+ '/'+ str(j)+ '.pgm' , 0) # '0' for reading grayscale images 
    x,y= img.shape
    vectorFlattened= img.reshape(x * y, 1).flatten()  
    if(j%2==1):
      training.append(vectorFlattened)
      trainingLabel.append(i)
      trainingLabel_Mix.append(1)
      new_TrainingLabel.append(1)
    else:
      testing.append(vectorFlattened)
      testingLabel.append(i)
      testingLabel_Mix.append(1)
      new_TestingLabel.append(1)  
    
    matrix.append(vectorFlattened)
    matrix_Mix.append(vectorFlattened)
    new_matrix.append(vectorFlattened)

DataMatrix= np.asarray(matrix)
training_Matrix= np.asarray(training)
testing_Matrix=np.asarray(testing)


# plt.imshow(img, cmap='gray')
# plt.axis('off')
# plt.title('A random grumpy person')
# plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


4) Classification using PCA

  a.Use the pseudo code below for computing the projection matrix U.
  
Define the alpha = {0.8,0.85,0.9,0.95}

In [ ]:

mean=np.mean(DataMatrix,axis=0)
centered=DataMatrix-mean
covarience=np.cov(training_Matrix.T)

EigenValues_PCA,EigenVectors_PCA= np.linalg.eigh(covarience)
print('Covariance Mat Shape:'+ str(covarience.shape))
indices = np.argsort(EigenValues_PCA)[::-1]
EigenValues_PCA = EigenValues_PCA[indices]
EigenVectors_PCA = EigenVectors_PCA[:,indices]
print('Eigen'+str(EigenValues_PCA))
Sum=sum(EigenValues_PCA)

List1 = []
List2 = []
List3 = []
List4 = []
prev=0
for k in range(len(EigenValues_PCA)):
  prev+=EigenValues_PCA[k]
  Alpha=(prev)/Sum
  if(Alpha<=0.8):   
    List1.append(EigenVectors_PCA[:,k])
  if(Alpha<=0.85):
    List2.append(EigenVectors_PCA[:,k])
  if(Alpha<=0.90):
    List3.append(EigenVectors_PCA[:,k])
  if(Alpha<=0.95):
    List4.append(EigenVectors_PCA[:,k])  
  else:
    break  

projection_matrix1=np.transpose(List1)
projection_matrix2=np.transpose(List2)
projection_matrix3=np.transpose(List3)
projection_matrix4=np.transpose(List4)



b. Project the training set, and test sets separately using the same
projection matrix.

In [ ]:
projectedtesting1=np.dot(testing_Matrix,projection_matrix1)
projectedtraining1=np.dot(training_Matrix,projection_matrix1)

projectedtesting2=np.dot(testing_Matrix,projection_matrix2)
projectedtraining2=np.dot(training_Matrix,projection_matrix2)

projectedtesting3=np.dot(testing_Matrix,projection_matrix3)
projectedtraining3=np.dot(training_Matrix,projection_matrix3)

projectedtesting4=np.dot(testing_Matrix,projection_matrix4)
projectedtraining4=np.dot(training_Matrix,projection_matrix4)


c. Use a simple classifier (first Nearest Neighbor to determine the class labels).

d. Report Accuracy for every value of alpha separately.

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining1,trainingLabel)

Testing_Label1 = classifier.predict(projectedtesting1)
#print(Testing_Label1)

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining2,trainingLabel)

Testing_Label2 = classifier.predict(projectedtesting2)
#print(Testing_Label2)
classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining3,trainingLabel)

Testing_Label3 = classifier.predict(projectedtesting3)
#print(Testing_Label3)

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining4,trainingLabel)

Testing_Label4 = classifier.predict(projectedtesting4)
#print(Testing_Label4)

Acc_score1 = accuracy_score(testingLabel, Testing_Label1)
print(Acc_score1)
Acc_score2 = accuracy_score(testingLabel, Testing_Label2)
print(Acc_score2)
Acc_score3 = accuracy_score(testingLabel, Testing_Label3)
print(Acc_score3)
Acc_score4 = accuracy_score(testingLabel, Testing_Label4)
print(Acc_score4)

5. Classification Using LDA 

a. Use the pseudo code below for LDA. We will modify few lines in
pseudocode to handle multiclass LDA.

i. Calculate the mean vector for every class Mu1, Mu2, ..., Mu40.

ii. Replace B matrix by Sb.

In [ ]:

MeanClass_Training= []

for i in range(0,200,5):
  newMatrix_Training=training_Matrix[i: i+5,:]
  M=np.mean(newMatrix_Training,axis=0)
  MeanClass_Training.append(M)
 
MeanClass_TrainingArray=np.asarray(MeanClass_Training)

B_MatrixTraining= np.zeros((10304, 10304))

print('Mean shape= '+str(mean.shape))

for j in range(40):
  MeanK=MeanClass_TrainingArray[j]-mean
  MeanKT=MeanK.T
  MeanProductK=5*(np.outer(MeanK,MeanKT))
  B_MatrixTraining= B_MatrixTraining + MeanProductK

# print('B_Matrix Training ='+str(B_MatrixTraining))



iii. S matrix remains the same, but it sums S1, S2, S3, ...S40.


In [ ]:
S_Matrix= np.zeros((10304,10304))

for i in range(40):
  centeredLda=np.subtract(training_Matrix[i],MeanClass_TrainingArray[i])
  centered_Transposed=centeredLda.T
  centered_lda_product=np.outer(centered_Transposed,centeredLda)
  S_Matrix=S_Matrix+centered_lda_product


# print('S_Matrix: '+ str(S_Matrix))
A = linalg.pinv(S_Matrix).dot(B_MatrixTraining)

iv. Use 39 dominant eigenvectors instead of just one. You will

have a projection matrix U39x10304.


In [ ]:

EigenValues_LDA , EigenVctors_LDA = np.linalg.eigh(A)
indices = np.argsort(EigenValues_LDA)[::-1]
EigenValues_LDA = EigenValues_LDA[indices]
EigenVctors_LDA = EigenVctors_LDA[:,indices]

DominantVectors = []

for i in range(39):

  DominantVectors.append(EigenVctors_LDA[:,i])

print('Size of Dom = '+ str(len(DominantVectors)))


b. Project the training set, and test sets separately using the same
projection matrix U. You will have 39 dimensions in the new space.


In [ ]:

LDA_ProjectionMatrix= np.transpose(DominantVectors)
LDA_Projected_Training= np.dot(training_Matrix,LDA_ProjectionMatrix)
LDA_Projected_Testing= np.dot(testing_Matrix,LDA_ProjectionMatrix)

c. Use a simple classifier (first Nearest Neighbor to determine the class labels).

d. Report Accuracy for the Multiclass LDA on the face recognition
dataset.

In [ ]:

classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(LDA_Projected_Training,trainingLabel)

Testing_Label_LDA = classifier.predict(LDA_Projected_Testing) 
# print(Testing_Label_LDA)

Acc_score_LDA = accuracy_score(testingLabel, Testing_Label_LDA)
print('Accuracy Score for LDA using 39 EigenVectors: ' + str(Acc_score_LDA))

6. Classifier Tuning 

a. Set the number of neighbors in the K-NN classifier to 1,3,5,7.
(Using PCA)

c. Plot (or tabulate) the performance measure (accuracy) against the K
value. This is to be done for PCA and LDA as well.

In [ ]:

kList = [1, 3, 5, 7]
#PCA
#Alpha = 0.80 K = 1

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining1,trainingLabel)

Label1_PCA = classifier.predict(projectedtesting1)
#print('PCA k=1 '+ str(Label1_PCA))

classifier = KNeighborsClassifier(n_neighbors=3,weights='distance')
classifier.fit(projectedtraining1,trainingLabel)

Label2_PCA= classifier.predict(projectedtesting1)
#print('PCA k=3 '+ str(Label2_PCA))

classifier = KNeighborsClassifier(n_neighbors=5,weights='distance')
classifier.fit(projectedtraining1,trainingLabel)

Label3_PCA = classifier.predict(projectedtesting1)
#print('PCA k=5 '+ str(Label3_PCA))

classifier = KNeighborsClassifier(n_neighbors=7,weights='distance')
classifier.fit(projectedtraining1,trainingLabel)

Label4_PCA = classifier.predict(projectedtesting1)
#print('PCA k=7 '+ str(Label4_PCA))

Acc_score_PCA_k1 = accuracy_score(testingLabel, Label1_PCA)
print('PCA k=1 Acc Score:'+str(Acc_score_PCA_k1))

Acc_score_PCA_k3 = accuracy_score(testingLabel, Label2_PCA)
print('PCA k=3 Acc Score:'+str(Acc_score_PCA_k3))

Acc_score_PCA_k5 = accuracy_score(testingLabel, Label3_PCA)
print('PCA k=5 Acc Score:'+str(Acc_score_PCA_k5))

Acc_score_PCA_k7 = accuracy_score(testingLabel, Label4_PCA)
print('PCA k=7 Acc Score:'+str(Acc_score_PCA_k7))

PCA_Accuracies = [Acc_score_PCA_k1, Acc_score_PCA_k3, Acc_score_PCA_k5, Acc_score_PCA_k7]
 
plt.figure(figsize=(5,5))
plt.plot(kList, PCA_Accuracies)
plt.xlabel('k value')
plt.ylabel('Accuracy')
plt.title('PCA Accuracy vs k value')
plt.show()


a. Set the number of neighbors in the K-NN classifier to 1,3,5,7.
(Using LDA)

c. Plot (or tabulate) the performance measure (accuracy) against the K
value. This is to be done for PCA and LDA as well.

In [ ]:

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(LDA_Projected_Training,trainingLabel)

Label1_LDA = classifier.predict(LDA_Projected_Testing) 
#print('LDA k=1 '+ str(Label1_LDA))

classifier = KNeighborsClassifier(n_neighbors=3,weights='distance')
classifier.fit(LDA_Projected_Training,trainingLabel)

Label2_LDA = classifier.predict(LDA_Projected_Testing) 
#print('LDA k=3 '+ str(Label2_LDA))

classifier = KNeighborsClassifier(n_neighbors=5,weights='distance')
classifier.fit(LDA_Projected_Training,trainingLabel)

Label3_LDA = classifier.predict(LDA_Projected_Testing) 
#print('LDA k=5 '+ str(Label3_LDA))

classifier = KNeighborsClassifier(n_neighbors=7,weights='distance')
classifier.fit(LDA_Projected_Training,trainingLabel)

Label4_LDA = classifier.predict(LDA_Projected_Testing) 
#print('LDA k=7 '+ str(Label4_LDA))

Acc_score_LDA_k1 = accuracy_score(testingLabel, Label1_LDA)
print('LDA k=1 Acc Score:'+str(Acc_score_LDA_k1))

Acc_score_LDA_k3 = accuracy_score(testingLabel, Label2_LDA)
print('LDA k=3 Acc Score:'+str(Acc_score_LDA_k3))

Acc_score_LDA_k5 = accuracy_score(testingLabel, Label3_LDA)
print('LDA k=5 Acc Score:'+str(Acc_score_LDA_k5))

Acc_score_LDA_k7 = accuracy_score(testingLabel, Label4_LDA)
print('LDA k=7 Acc Score:'+str(Acc_score_LDA_k7))

plt.figure(figsize=(5,5))
LDA_Accuracies = [Acc_score_LDA_k1, Acc_score_LDA_k3, Acc_score_LDA_k5, Acc_score_LDA_k7]
plt.plot(kList, LDA_Accuracies)
plt.xlabel('k value')
plt.ylabel('Accuracy')
plt.title('LDA Accuracy vs k value')
plt.show()

7. Compare vs Non-Face Images 

a. Download non-face images and make them of the same size 92x112.
and try to solve the classification problem faces vs. Non-faces.

In [ ]:

training_Mix = training
testing_Mix = testing
#matrix_Mix=new_matrix#matrix#either 400 and 400 or 200 and 200 total matrix

# print('Lenghth of training Face: '+ str(len(training_Mix)))
# print('Lenghth of matrix Face: '+ str(len(matrix_Mix)))

for k in range(400):

    if(k<10):
      img2 = cv2.imread(Base2 + '/AnyConv.com__flower_000'+ str(k)+ '.pgm' , 0)
      img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
    elif(k<100):
      img2 = cv2.imread(Base2 + '/AnyConv.com__flower_00'+ str(k)+ '.pgm' , 0)
      img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
    else:
      img2 = cv2.imread(Base2 + '/AnyConv.com__flower_0'+ str(k)+ '.pgm' , 0)
      img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)

    x2,y2= img2_resize.shape
    vectorFlattened2= img2_resize.reshape(x2 * y2, 1).flatten()  
    if(k%2==1):
      training_Mix.append(vectorFlattened2)
      trainingLabel_Mix.append(2)
    else:
      testing_Mix.append(vectorFlattened2)
      testingLabel_Mix.append(2)  
   
    matrix_Mix.append(vectorFlattened2)

# print('len of Training = '+str(len(training_Mix)))
DataMatrix_Mix=np.asarray(matrix_Mix)
training_Matrix_Mix= np.asarray(training_Mix)
testing_Matrix_Mix=np.asarray(testing_Mix)
# print(trainingLabel_Mix)
# print(img2_resize.shape)
# print(DataMatrix_Mix.shape)

# plt.imshow(img2, cmap='gray')
# plt.axis('off')
# plt.title('Flower img')
# plt.show()
    

i. Show failure and success cases.

ii. How many dominant eigenvectors will you use for the LDA
solution?  (We Used Eigen Vectors using default alpha = 0.95)

In [ ]:
Mean_Mix=np.mean(DataMatrix_Mix,axis=0)
MeanClass_TrainingMix= []

# Mean of first class (Faces)
FaceMatrix_Training=training_Matrix_Mix[0:200,:]
M1=np.mean(FaceMatrix_Training,axis=0)
MeanClass_TrainingMix.append(M1)

#Mean of second class (Non-Faces)
NonFaceMatrix_Training=training_Matrix_Mix[200:400,:]
M2=np.mean(NonFaceMatrix_Training,axis=0)
MeanClass_TrainingMix.append(M2)
 
MeanClass_TrainingArrayMix=np.asarray(MeanClass_TrainingMix)
#print('meanclassmix'+str(MeanClass_TrainingArrayMix.shape))

B_MatrixTraining_Mix= np.zeros((10304, 10304))

#print('Mean Class Training Array shape= '+str(MeanClass_TrainingArrayMix.shape))

for j in range(2):
  MeanK=MeanClass_TrainingArrayMix[j]-Mean_Mix
  MeanKT=MeanK.T
  MeanProductK=2*(np.outer(MeanK,MeanKT))
  B_MatrixTraining_Mix= B_MatrixTraining_Mix + MeanProductK

#print('B_Matrix Training ='+str(B_MatrixTraining_Mix.shape))

S_Matrix_Mix= np.zeros((10304,10304))

for i in range(2):
  centeredLda=np.subtract(training_Matrix_Mix[i],MeanClass_TrainingArrayMix[i])
  centered_Transposed=centeredLda.T
  centered_lda_product=np.outer(centered_Transposed,centeredLda)
  S_Matrix_Mix=S_Matrix_Mix+centered_lda_product


#print('S_Matrix: '+ str(S_Matrix_Mix.shape))

A = linalg.pinv(S_Matrix_Mix).dot(B_MatrixTraining_Mix)


EigenValuesMix_LDA , EigenVectorsMix_LDA = np.linalg.eigh(A)
indices = np.argsort(EigenValuesMix_LDA)[::-1]
EigenValuesMix_LDA = EigenValuesMix_LDA[indices]
EigenVectorsMix_LDA = EigenVectorsMix_LDA[:,indices]

Sum=sum(EigenValuesMix_LDA)

List = []

prev1=0
for k in range(len(EigenValuesMix_LDA)):
  prev1+=EigenValuesMix_LDA[k]
  Alpha_Mix=(prev1)/Sum
  if(Alpha_Mix<=0.95):   
    List.append(EigenVectorsMix_LDA[:,k])
  else:
    break  

# print('Eignvectors length'+str(len(List)))
#print('Eignvectors '+str(List))

LDA_ProjectionMatrix_Mix= np.transpose(List)
LDA_Projected_Training_Mix= np.dot(training_Matrix_Mix,LDA_ProjectionMatrix_Mix)
LDA_Projected_Testing_Mix= np.dot(testing_Matrix_Mix,LDA_ProjectionMatrix_Mix)


classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(LDA_Projected_Training_Mix,trainingLabel_Mix)

Testing_Label_LDA_Mix = classifier.predict(LDA_Projected_Testing_Mix) 
print('testingLabelMix'+str(Testing_Label_LDA_Mix))

Acc_score_LDA_Mix = accuracy_score(testingLabel_Mix, Testing_Label_LDA_Mix)
print('LDA Accuracy Faces vs Non Faces '+str(Acc_score_LDA_Mix))

Failure=[]
Success=[]
for z in range(400):
  if(z<200):
    if(Testing_Label_LDA_Mix[z]==2):
      Failure.append(LDA_Projected_Training_Mix[z])
    else:
      Success.append(LDA_Projected_Training_Mix[z])
  else:
    if(Testing_Label_LDA_Mix[z]==1):
      Failure.append(LDA_Projected_Training_Mix[z])
    else:
      Success.append(LDA_Projected_Training_Mix[z])

print('Number of Failure Cases = '+str(len(Failure)))
print('Number of Success Cases = '+str(len(Success)))

iii. Plot the accuracy vs the number of non-faces images while fixing
the number of face images.


In [ ]:

new_Training = training
new_Testing = testing
AccuracyScoreList = []
NonFace_number = [] 

for k in range(400):
  if(k<10):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_000'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  elif(k<100):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_00'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  else:
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_0'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)

  x2,y2= img2_resize.shape
  vectorFlattened2= img2_resize.reshape(x2 * y2, 1).flatten()  
  if(k%2==1):
    new_Training.append(vectorFlattened2)
    new_TrainingLabel.append(2)
  else:
    new_Testing.append(vectorFlattened2)
    new_TestingLabel.append(2)   
  
  new_matrix.append(vectorFlattened2)

  if(k==99 or k==199 or k==299 or k==399):
    print('Training Label size = '+str(len(new_TrainingLabel)))
    NonFace_number.append(k)
    new_DataMatrix=np.asarray(new_matrix)
    new_Mean=np.mean(new_DataMatrix,axis=0)
    new_training_Matrix= np.asarray(new_Training)
    new_testing_Matrix=np.asarray(new_Testing)
    # print(new_TrainingLabel)
    # print(img2_resize.shape)
    # print(new_DataMatrix.shape)


    new_MeanClass_Training= []

    # Mean of first class (Faces)
    new_FaceMatrix_Training=new_training_Matrix[0:200,:]
    new_M1=np.mean(new_FaceMatrix_Training,axis=0)
    new_MeanClass_Training.append(new_M1)

    #Mean of second class (Non-Faces)
    new_NonFaceMatrix_Training=new_training_Matrix[200:400,:]
    new_M2=np.mean(new_NonFaceMatrix_Training,axis=0)
    new_MeanClass_Training.append(new_M2)

    new_MeanClass_TrainingArray=np.asarray(new_MeanClass_Training)
    #print('meanclassmix'+str(new_MeanClass_TrainingArray.shape))

    new_B_MatrixTraining= np.zeros((10304, 10304))

    #print('Mean Class Training Array shape= '+str(new_MeanClass_TrainingArray.shape))

    for s in range(2):
      new_MeanK=new_MeanClass_TrainingArray[s]-new_Mean
      new_MeanKT=new_MeanK.T
      new_MeanProductK=2*(np.outer(new_MeanK,new_MeanKT))
      new_B_MatrixTraining= new_B_MatrixTraining + new_MeanProductK

    #print('B_Matrix Training ='+str(new_B_MatrixTraining.shape))

    new_S_Matrix= np.zeros((10304,10304))

    for m in range(2):
      centeredLda=np.subtract(new_training_Matrix[m],new_MeanClass_TrainingArray[m])
      centered_Transposed=centeredLda.T
      centered_lda_product=np.outer(centered_Transposed,centeredLda)
      new_S_Matrix=new_S_Matrix+centered_lda_product


    #print('S_Matrix: '+ str(new_S_Matrix.shape))

    new_A = np.linalg.inv(new_S_Matrix).dot(new_B_MatrixTraining)

    new_EigenValues_LDA , new_EigenVectors_LDA = np.linalg.eigh(new_A)
    indices = np.argsort(new_EigenValues_LDA)[::-1]
    new_EigenValues_LDA = new_EigenValues_LDA[indices]
    new_EigenVectors_LDA = new_EigenVectors_LDA[:,indices]

    DominantVectors_Q7 = []
    for r in range(39):
      DominantVectors_Q7.append(new_EigenVectors_LDA[:,r])

    print('Size of Dom = '+ str(len(DominantVectors_Q7)))

    print('Eignvectors length Dom'+str(len(DominantVectors_Q7)))
    new_LDA_ProjectionMatrix= np.transpose(DominantVectors_Q7)
    new_LDA_Projected_Training= np.dot(new_training_Matrix,new_LDA_ProjectionMatrix)
    new_LDA_Projected_Testing= np.dot(new_testing_Matrix,new_LDA_ProjectionMatrix)


    classifier = KNeighborsClassifier(n_neighbors=1)
    classifier.fit(new_LDA_Projected_Training,new_TrainingLabel)

    new_Testing_Label_LDA = classifier.predict(new_LDA_Projected_Testing) 
    #print('testingLabelMix'+str(new_Testing_Label_LDA))

    new_Acc_score_LDA = accuracy_score(new_TestingLabel, new_Testing_Label_LDA)
    print('Accuracymix'+str(new_Acc_score_LDA))
    AccuracyScoreList.append(new_Acc_score_LDA)
  
    
plt.figure(figsize=(5,5))
plt.plot(NonFace_number, AccuracyScoreList)
plt.xlabel('NonFace number')
plt.ylabel('Accuracy')
plt.title('NonFace number vs Accuracy')
plt.show()

8. Bonus


In [ ]:
DataMatrix_Label=[]
x_train=[]
x_test=[]
y_train=[]
y_test=[]
for i in range(1,41):
  for j in range(1,11):
    img = cv2.imread(BASE + '/s'+ str(i)+ '/'+ str(j)+ '.pgm' , 0) # '0' for reading grayscale images 
    x,y= img.shape
    vectorFlattened= img.reshape(x * y, 1).flatten()
    if(j<=7):
      x_train.append(vectorFlattened)
      y_train.append(i)
      trainingLabel_Mix.append(1)
    else:
      x_test.append(vectorFlattened)
      y_test.append(i)
    DataMatrix_Label.append(i)  

#x_train, x_test,y_train,y_test = train_test_split(DataMatrix,DataMatrix_Label,test_size=0.30)
print(y_train)
print(y_test)
training_Matrix_new= np.asarray(x_train)
testing_Matrix_new=np.asarray(x_test)

#Question 4 PCA

covarience_new=np.cov(training_Matrix_new.T)
EigenValues_PCA_new,EigenVectors_PCA_new= np.linalg.eigh(covarience_new)
print('Covariance Mat new Shape:'+ str(covarience_new.shape))
indices = np.argsort(EigenValues_PCA_new)[::-1]
EigenValues_PCA_new = EigenValues_PCA_new[indices]
EigenVectors_PCA_new = EigenVectors_PCA_new[:,indices]
print('Eigen new'+str(EigenValues_PCA_new))
Sum_new=sum(EigenValues_PCA_new)

List1_new = []
List2_new = []
List3_new = []
List4_new = []
new=0
for k in range(len(EigenValues_PCA_new)):
  new+=EigenValues_PCA_new[k]
  Alpha_new=(new)/Sum_new
  if(Alpha_new<=0.8):   
    List1_new.append(EigenVectors_PCA_new[:,k])
  if(Alpha_new<=0.85):
    List2_new.append(EigenVectors_PCA_new[:,k])
  if(Alpha_new<=0.90):
    List3_new.append(EigenVectors_PCA_new[:,k])
  if(Alpha_new<=0.95):
    List4_new.append(EigenVectors_PCA_new[:,k])  
  else:
    break  

projection_matrix1_new=np.transpose(List1_new)
projection_matrix2_new=np.transpose(List2_new)
projection_matrix3_new=np.transpose(List3_new)
projection_matrix4_new=np.transpose(List4_new)


projectedtesting1_new=np.dot(testing_Matrix_new,projection_matrix1_new)
projectedtraining1_new=np.dot(training_Matrix_new,projection_matrix1_new)

projectedtesting2_new=np.dot(testing_Matrix_new,projection_matrix2_new)
projectedtraining2_new=np.dot(training_Matrix_new,projection_matrix2_new)

projectedtesting3_new=np.dot(testing_Matrix_new,projection_matrix3_new)
projectedtraining3_new=np.dot(training_Matrix_new,projection_matrix3_new)

projectedtesting4_new=np.dot(testing_Matrix_new,projection_matrix4_new)
projectedtraining4_new=np.dot(training_Matrix_new,projection_matrix4_new)

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining1_new,y_train)

Testing_Label1_new = classifier.predict(projectedtesting1_new)



classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining2_new,y_train)

Testing_Label2_new = classifier.predict(projectedtesting2_new)
#print(Testing_Label2)

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining3_new,y_train)

Testing_Label3_new = classifier.predict(projectedtesting3_new)
#print(Testing_Label3)

classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining4_new,y_train)

Testing_Label4_new = classifier.predict(projectedtesting4_new)
#print(Testing_Label4_new)


Acc_score1_new =accuracy_score(y_test, Testing_Label1_new)
print('Accuracy of Alpha=0.80: '+str(Acc_score1_new))
Acc_score2_new = accuracy_score(y_test, Testing_Label2_new)
print('Accuracy of Alpha=0.85: '+str(Acc_score2_new))
Acc_score3_new = accuracy_score(y_test, Testing_Label3_new)
print('Accuracy of Alpha=0.90: '+str(Acc_score3_new))
Acc_score4_new = accuracy_score(y_test, Testing_Label4_new)
print('Accuracy of Alpha=0.95: '+str(Acc_score4_new))

# #Question 5 LDA

MeanClass_Training_new= []

for i in range(0,280,7):
  newMatrix_Training_new=training_Matrix_new[i: i+7,:]
  M_new=np.mean(newMatrix_Training_new,axis=0)
  MeanClass_Training_new.append(M_new)
 
MeanClass_TrainingArray_new=np.asarray(MeanClass_Training_new)

B_MatrixTraining_new= np.zeros((10304, 10304))

for j in range(40):
  MeanK_new=MeanClass_TrainingArray_new[j]-mean
  MeanKT_new=MeanK_new.T
  MeanProductK_new=7*(np.outer(MeanK_new,MeanKT_new))
  B_MatrixTraining_new= B_MatrixTraining_new + MeanProductK_new

#print('B_Matrix Training New='+str(B_MatrixTraining_new))

S_Matrix_new= np.zeros((10304,10304))

for i in range(40):
  newMatrix_Training_new=training_Matrix_new[i: i+7,:]
  centeredLda_new=np.subtract(newMatrix_Training_new,MeanClass_TrainingArray_new[i])
  centered_Transposed_new=centeredLda_new.T
  centered_lda_product_new=np.dot(centered_Transposed_new,centeredLda_new)
  S_Matrix_new=S_Matrix_new+centered_lda_product_new


# #print('S_Matrix new: '+ str(S_Matrix_new))
A_new = pinvh(S_Matrix_new).dot(B_MatrixTraining_new)

EigenValues_LDA_new , EigenVctors_LDA_new = np.linalg.eigh(A_new)
indices = np.argsort(EigenValues_LDA_new)[::-1]
EigenValues_LDA_new = EigenValues_LDA_new[indices]
EigenVctors_LDA_new = EigenVctors_LDA_new[:,indices]

DominantVectors_new = []

for i in range(39):

  DominantVectors_new.append(EigenVctors_LDA_new[:,i])


LDA_ProjectionMatrix_new= np.transpose(DominantVectors_new)
LDA_Projected_Training_new= np.dot(training_Matrix_new,LDA_ProjectionMatrix_new)
LDA_Projected_Testing_new= np.dot(testing_Matrix_new,LDA_ProjectionMatrix_new)


classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(LDA_Projected_Training_new,y_train)

Testing_Label_LDA_new = classifier.predict(LDA_Projected_Testing_new) 

Acc_score_LDA_new = accuracy_score(y_test, Testing_Label_LDA_new)
print('Accuracy using LDA'+str(Acc_score_LDA_new))

Question 6 Classifier Tuning
kList = [1, 3, 5, 7]
#PCA
#Alpha = 0.85 K = 1
classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(projectedtraining4_new,y_train)

Label1_PCA_new = classifier.predict(projectedtesting4_new)
#print('PCA k=1 '+ str(Label1_PCA_new))

classifier = KNeighborsClassifier(n_neighbors=3,weights='distance')
classifier.fit(projectedtraining4_new,y_train)

Label2_PCA_new= classifier.predict(projectedtesting4_new)
#print('PCA k=3 '+ str(Label2_PCA))

classifier = KNeighborsClassifier(n_neighbors=5,weights='distance')
classifier.fit(projectedtraining4_new,y_train)

Label3_PCA_new = classifier.predict(projectedtesting4_new)
#print('PCA k=5 '+ str(Label3_PCA))

classifier = KNeighborsClassifier(n_neighbors=7,weights='distance')
classifier.fit(projectedtraining4_new,y_train)

Label4_PCA_new = classifier.predict(projectedtesting4_new)
#print('PCA k=7 '+ str(Label4_PCA))


Acc_score_PCA_k1_new = accuracy_score(y_test, Label1_PCA_new)
print('PCA k=1 Acc Score:'+str(Acc_score_PCA_k1_new))

Acc_score_PCA_k3_new = accuracy_score(y_test, Label2_PCA_new)
print('PCA k=3 Acc Score:'+str(Acc_score_PCA_k3_new))

Acc_score_PCA_k5_new = accuracy_score(y_test, Label3_PCA_new)
print('PCA k=5 Acc Score:'+str(Acc_score_PCA_k5_new))

Acc_score_PCA_k7_new = accuracy_score(y_test, Label4_PCA_new)
print('PCA k=7 Acc Score:'+str(Acc_score_PCA_k7_new))

PCA_Accuracies_new = [Acc_score_PCA_k1_new, Acc_score_PCA_k3_new, Acc_score_PCA_k5_new, Acc_score_PCA_k7_new]
 
plt.figure(figsize=(5,5))
plt.plot(kList, PCA_Accuracies_new)
plt.xlabel('k value')
plt.ylabel('Accuracy')
plt.title('PCA Accuracy vs k value')
plt.show()

#LDA
classifier = KNeighborsClassifier(n_neighbors=1,weights='distance')
classifier.fit(LDA_Projected_Training_new,y_train)

Label1_LDA_new = classifier.predict(LDA_Projected_Testing_new) 
#print('LDA k=1 '+ str(Label1_LDA_new))

classifier = KNeighborsClassifier(n_neighbors=3,weights='distance')
classifier.fit(LDA_Projected_Training_new,y_train)

Label2_LDA_new = classifier.predict(LDA_Projected_Testing_new) 
#print('LDA k=3 '+ str(Label2_LDA_new))

classifier = KNeighborsClassifier(n_neighbors=5,weights='distance')
classifier.fit(LDA_Projected_Training_new,y_train)

Label3_LDA_new = classifier.predict(LDA_Projected_Testing_new) 
#print('LDA k=5 '+ str(Label3_LDA_new))

classifier = KNeighborsClassifier(n_neighbors=7,weights='distance')
classifier.fit(LDA_Projected_Training_new,y_train)

Label4_LDA_new = classifier.predict(LDA_Projected_Testing_new) 
#print('LDA k=7 '+ str(Label4_LDA_new))

Acc_score_LDA_k1_new = accuracy_score(y_test, Label1_LDA_new)
print('LDA k=1 Acc Score:'+str(Acc_score_LDA_k1_new))

Acc_score_LDA_k3_new = accuracy_score(y_test, Label2_LDA_new)
print('LDA k=3 Acc Score:'+str(Acc_score_LDA_k3_new))

Acc_score_LDA_k5_new = accuracy_score(y_test, Label3_LDA_new)
print('LDA k=5 Acc Score:'+str(Acc_score_LDA_k5_new))

Acc_score_LDA_k7_new = accuracy_score(y_test, Label4_LDA_new)
print('LDA k=7 Acc Score:'+str(Acc_score_LDA_k7_new))

plt.figure(figsize=(5,5))
LDA_Accuracies_new = [Acc_score_LDA_k1_new, Acc_score_LDA_k3_new, Acc_score_LDA_k5_new, Acc_score_LDA_k7_new]
plt.plot(kList, LDA_Accuracies_new)
plt.xlabel('k value')
plt.ylabel('Accuracy')
plt.title('LDA Accuracy vs k value')
plt.show()

# # Question 7
training_Mix = x_train
testing_Mix = x_test

#matrix_Mix=new_matrix#matrix#either 400 and 400 or 200 and 200 total matrix

print('Lenghth of training Face: '+ str(len(training_Mix)))
print('Lenghth of matrix Face: '+ str(len(matrix_Mix)))
for k in range(400):

  if(k<10):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_000'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  elif(k<100):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_00'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  else:
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_0'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)

  x2,y2= img2_resize.shape
  vectorFlattened2= img2_resize.reshape(x2 * y2, 1).flatten()  
  if(k<=7):
    training_Mix.append(vectorFlattened2)
    trainingLabel_Mix.append(2)
  else:
    testing_Mix.append(vectorFlattened2)
    testingLabel_Mix.append(2)  
   
  matrix_Mix.append(vectorFlattened2)



print('len of Training = '+str(len(training_Mix)))
DataMatrix_Mix=np.asarray(matrix_Mix)
Mean_Mix=np.mean(DataMatrix_Mix,axis=0)
training_Matrix_Mix= np.asarray(training_Mix)
testing_Matrix_Mix=np.asarray(testing_Mix)
print(trainingLabel_Mix)
print(img2_resize.shape)
print(DataMatrix_Mix.shape)
    

MeanClass_TrainingMix= []

# Mean of first class (Faces)
FaceMatrix_Training=training_Matrix_Mix[0:200,:]
M1=np.mean(FaceMatrix_Training,axis=0)
MeanClass_TrainingMix.append(M1)

#Mean of second class (Non-Faces)
NonFaceMatrix_Training=training_Matrix_Mix[200:400,:]
M2=np.mean(NonFaceMatrix_Training,axis=0)
MeanClass_TrainingMix.append(M2)
 
MeanClass_TrainingArrayMix=np.asarray(MeanClass_TrainingMix)
#print('meanclassmix'+str(MeanClass_TrainingArrayMix.shape))

B_MatrixTraining_Mix= np.zeros((10304, 10304))

#print('Mean Class Training Array shape= '+str(MeanClass_TrainingArrayMix.shape))

for j in range(2):
  MeanK=MeanClass_TrainingArrayMix[j]-Mean_Mix
  MeanKT=MeanK.T
  MeanProductK=2*(np.outer(MeanK,MeanKT))
  B_MatrixTraining_Mix= B_MatrixTraining_Mix + MeanProductK

#print('B_Matrix Training ='+str(B_MatrixTraining_Mix.shape))

S_Matrix_Mix= np.zeros((10304,10304))

for i in range(2):
  centeredLda=np.subtract(training_Matrix_Mix[i],MeanClass_TrainingArrayMix[i])
  centered_Transposed=centeredLda.T
  centered_lda_product=np.outer(centered_Transposed,centeredLda)
  S_Matrix_Mix=S_Matrix_Mix+centered_lda_product


#print('S_Matrix: '+ str(S_Matrix_Mix.shape))

A = np.linalg.inv(S_Matrix_Mix).dot(B_MatrixTraining_Mix)

EigenValuesMix_LDA , EigenVectorsMix_LDA = np.linalg.eigh(A)
indices = np.argsort(EigenValuesMix_LDA)[::-1]
EigenValuesMix_LDA = EigenValuesMix_LDA[indices]
EigenVectorsMix_LDA = EigenVectorsMix_LDA[:,indices]

Sum=sum(EigenValuesMix_LDA)

List = []

prev1=0
for k in range(len(EigenValuesMix_LDA)):
  prev1+=EigenValuesMix_LDA[k]
  Alpha_Mix=(prev1)/Sum
  if(Alpha_Mix<=0.95):   
    List.append(EigenVectorsMix_LDA[:,k])
  else:
    break  

print('Eignvectors length'+str(len(List)))
#print('Eignvectors '+str(List))

LDA_ProjectionMatrix_Mix= np.transpose(List)
LDA_Projected_Training_Mix= np.dot(training_Matrix_Mix,LDA_ProjectionMatrix_Mix)
LDA_Projected_Testing_Mix= np.dot(testing_Matrix_Mix,LDA_ProjectionMatrix_Mix)


classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(LDA_Projected_Training_Mix,trainingLabel_Mix)

Testing_Label_LDA_Mix = classifier.predict(LDA_Projected_Testing_Mix) 
print('testingLabelMix'+str(Testing_Label_LDA_Mix))

Acc_score_LDA_Mix = accuracy_score(testingLabel_Mix, Testing_Label_LDA_Mix)
print('Accuracymix'+str(Acc_score_LDA_Mix))
Failure=[]
Success=[]
for z in range(400):
  if(z<200):
    if(Testing_Label_LDA_Mix[z]==2):
      Failure.append(LDA_Projected_Training_Mix[z])
    else:
      Success.append(LDA_Projected_Training_Mix[z])
  else:
    if(Testing_Label_LDA_Mix[z]==1):
      Failure.append(LDA_Projected_Training_Mix[z])
    else:
      Success.append(LDA_Projected_Training_Mix[z])

print('Number of Failure Cases = '+str(len(Failure)))
print('Number of Success Cases = '+str(len(Success)))

# Question 7 iii
new_Training = x_train
new_Testing = x_test
AccuracyScoreList = []
NonFace_number = [] 

for k in range(400):
  if(k<10):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_000'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  elif(k<100):
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_00'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)
  else:
    img2 = cv2.imread(Base2 + '/AnyConv.com__flower_0'+ str(k)+ '.pgm' , 0)
    img2_resize=cv2.resize(img2,(112,92),interpolation=cv2.INTER_AREA)

  x2,y2= img2_resize.shape
  vectorFlattened2= img2_resize.reshape(x2 * y2, 1).flatten()  
  if(k<=7):
    new_Training.append(vectorFlattened2)
    new_TrainingLabel.append(2)
  else:
    new_Testing.append(vectorFlattened2)
    new_TestingLabel.append(2)   
  
  new_matrix.append(vectorFlattened2)

  if(k==99 or k==199 or k==299 or k==399):
    print('Training Label size = '+str(len(new_TrainingLabel)))
    NonFace_number.append(k)
    new_DataMatrix=np.asarray(new_matrix)
    new_Mean=np.mean(new_DataMatrix,axis=0)
    new_training_Matrix= np.asarray(new_Training)
    new_testing_Matrix=np.asarray(new_Testing)
    #print(new_TrainingLabel)
    print(img2_resize.shape)
    print(new_DataMatrix.shape)


    new_MeanClass_Training= []

    # Mean of first class (Faces)
    new_FaceMatrix_Training=new_training_Matrix[0:200,:]
    new_M1=np.mean(new_FaceMatrix_Training,axis=0)
    new_MeanClass_Training.append(new_M1)

    #Mean of second class (Non-Faces)
    new_NonFaceMatrix_Training=new_training_Matrix[200:400,:]
    new_M2=np.mean(new_NonFaceMatrix_Training,axis=0)
    new_MeanClass_Training.append(new_M2)

    new_MeanClass_TrainingArray=np.asarray(new_MeanClass_Training)
    #print('meanclassmix'+str(new_MeanClass_TrainingArray.shape))

    new_B_MatrixTraining= np.zeros((10304, 10304))

    #print('Mean Class Training Array shape= '+str(new_MeanClass_TrainingArray.shape))

    for s in range(2):
      new_MeanK=new_MeanClass_TrainingArray[s]-new_Mean
      new_MeanKT=new_MeanK.T
      new_MeanProductK=2*(np.outer(new_MeanK,new_MeanKT))
      new_B_MatrixTraining= new_B_MatrixTraining + new_MeanProductK

    #print('B_Matrix Training ='+str(new_B_MatrixTraining.shape))

    new_S_Matrix= np.zeros((10304,10304))

    for m in range(2):
      centeredLda=np.subtract(new_training_Matrix[m],new_MeanClass_TrainingArray[m])
      centered_Transposed=centeredLda.T
      centered_lda_product=np.outer(centered_Transposed,centeredLda)
      new_S_Matrix=new_S_Matrix+centered_lda_product


    #print('S_Matrix: '+ str(new_S_Matrix.shape))

    new_A = np.linalg.inv(new_S_Matrix).dot(new_B_MatrixTraining)

    new_EigenValues_LDA , new_EigenVectors_LDA = np.linalg.eigh(new_A)
    indices = np.argsort(new_EigenValues_LDA)[::-1]
    new_EigenValues_LDA = new_EigenValues_LDA[indices]
    new_EigenVectors_LDA = new_EigenVectors_LDA[:,indices]

    DominantVectors_Q7 = []
    for r in range(39):
      DominantVectors_Q7.append(new_EigenVectors_LDA[:,r])

    print('Size of Dom = '+ str(len(DominantVectors_Q7)))

    print('Eignvectors length Dom'+str(len(DominantVectors_Q7)))
    new_LDA_ProjectionMatrix= np.transpose(DominantVectors_Q7)
    new_LDA_Projected_Training= np.dot(new_training_Matrix,new_LDA_ProjectionMatrix)
    new_LDA_Projected_Testing= np.dot(new_testing_Matrix,new_LDA_ProjectionMatrix)


    classifier = KNeighborsClassifier(n_neighbors=1)
    classifier.fit(new_LDA_Projected_Training,new_TrainingLabel)

    new_Testing_Label_LDA = classifier.predict(new_LDA_Projected_Testing) 
    #print('testingLabelMix'+str(new_Testing_Label_LDA))

    new_Acc_score_LDA = accuracy_score(new_TestingLabel, new_Testing_Label_LDA)
    print('Accuracymix'+str(new_Acc_score_LDA))
    AccuracyScoreList.append(new_Acc_score_LDA)
    
plt.figure(figsize=(5,5))
plt.plot(NonFace_number, AccuracyScoreList)
plt.xlabel('NonFace number')
plt.ylabel('Accuracy')
plt.title('NonFace number vs Accuracy')
plt.show()

[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38

KeyboardInterrupt: ignored

# New Section